In [ ]:
import utils
utils.create_unique_folder("test")
utils.create_unique_folder("test")


In [ ]:
from utils import load_model
from TCN import TCNAE

path = "HyperparameterSearchAllModels/Models/TCNAE_472.pth"
paras = {"seq_len": 61, "input_dim": 1, "num_channels": [8, 16], "kernel_size": 7, "latent_dim": 10}
model = load_model(path, TCNAE, paras)
model

In [ ]:
from utils import load_np_dataset
import torch
import numpy as np
fps = 90
cond = "nod"
f = 62
b = 500
a = 200
angles_correct = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
angles_incorrect = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
names_correct = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
names_incorrect = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
pat_names = np.unique(names_correct)
n = int(0.7 * len(pat_names))
train_pats = pat_names[:n]
test_pats = pat_names[n:]
train_data = angles_correct[np.isin(names_correct, train_pats)]
test_correct = angles_correct[np.isin(names_correct, test_pats)]
test_incorrect = angles_incorrect[np.isin(names_incorrect, test_pats)]

test_correct = torch.tensor(test_correct).type(torch.float)[:, None, :]
test_incorrect = torch.tensor(test_incorrect).type(torch.float)[:, None, :]
train_correct= torch.tensor(train_data).type(torch.float)[:, None, :]
model.eval()
with torch.no_grad():
    train_correct_out = model(train_correct)
    correct_out = model(test_correct)
    incorrect_out = model(test_incorrect)
    mse_correct = torch.mean((correct_out - test_correct)**2, dim=(1, 2))
    mse_incorrect = torch.mean((incorrect_out - test_incorrect)**2, dim=(1, 2))
    mse_train_correct = torch.mean((train_correct_out - train_correct)**2, dim=(1, 2))
    print(mse_correct)
    print(mse_incorrect)
    print(mse_train_correct)

In [ ]:
import matplotlib.pyplot as plt
perc = np.linspace(80, 100, 100)
ths = np.percentile(mse_train_correct.cpu().numpy(), perc)
fig, ax = plt.subplots()
res = {"TH_number": list(), "TH": list(), "Correct": list(), "Incorrect": list()}
for p, th in zip(perc, ths):
    correct_acc = np.mean((mse_correct < th).cpu().numpy())
    incorrect_acc = np.mean((mse_incorrect > th).cpu().numpy())
    res["TH_number"].append(th)
    res["TH"].append(p)
    res["Correct"].append(correct_acc)
    res["Incorrect"].append(incorrect_acc)

ax.plot(res["TH"], res["Correct"], label="Correct", color="b")
ax.plot(res["TH"], res["Incorrect"], label="Incorrect", color="orange")
ax.axhline(0.9, linestyle="--", color="red", alpha=0.4)
ax.set_ylabel("Recall")
ax.set_xlabel("Percentile")
ax.legend()
plt.show()

In [ ]:
from utils import load_model
from TCN import TCNAE
import torch
import numpy as np

model_parameter = {"seq_len":43, "input_dim":1, "num_channels": [8, 16], "kernel_size":3, "latent_dim":10}
model = load_model("Results/LOSO_500msSeq/Models/TCNAE_12_nod.pth", TCNAE, model_parameter)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

fps = 90
cond = "nod"
f = 44
b = 300
a = 200
angles_correct = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
angles_incorrect = np.load(f"../Data/Dataset_Prepare/angles_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
names_correct = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Correct_f{f}_b{b}_a{a}.npy")
names_incorrect = np.load(f"../Data/Dataset_Prepare/names_fps{fps}_{cond}_Incorrect_f{f}_b{b}_a{a}.npy")
pat_names = np.unique(names_correct)
n = int(0.7 * len(pat_names))
train_pats = pat_names[:n]
test_pats = pat_names[n:]
train_data = angles_correct[np.isin(names_correct, train_pats)]
test_correct = angles_correct[np.isin(names_correct, test_pats)]
test_incorrect = angles_incorrect[np.isin(names_incorrect, test_pats)]

test_correct = torch.tensor(test_correct).type(torch.float)[:, None, :]
test_incorrect = torch.tensor(test_incorrect).type(torch.float)[:, None, :]
train_correct= torch.tensor(train_data).type(torch.float)[:, None, :]
model.eval()
with torch.no_grad():
    train_correct_out = model(train_correct)
    correct_out = model(test_correct)
    incorrect_out = model(test_incorrect)
    mse_correct = torch.mean((correct_out - test_correct)**2, dim=(1, 2))
    mse_incorrect = torch.mean((incorrect_out - test_incorrect)**2, dim=(1, 2))
    mse_train_correct = torch.mean((train_correct_out - train_correct)**2, dim=(1, 2))
    print(mse_correct)
    print(mse_incorrect)
    print(mse_train_correct)

In [ ]:
import matplotlib.pyplot as plt
trial_nr = 1
correct = {"in": test_correct[trial_nr][0].numpy(), "out": correct_out[trial_nr][0].numpy()}
incorrect = {"in": test_incorrect[trial_nr][0].numpy(), "out": incorrect_out[trial_nr][0].numpy()}

fig, (ax_correct, ax_incorrect) = plt.subplots(2, sharex=True)
ax_correct.plot(correct["in"], color="blue")
ax_correct.set_title("Correct")
ax_incorrect.plot(incorrect["in"], color="blue")
ax_incorrect.set_title("Incorrect")
fig.tight_layout()
plt.show()

mse_correct = np.mean((correct["in"] - correct["out"])**2)
mse_incorrect = np.mean((incorrect["in"] - incorrect["out"])**2)

fig, (ax_correct, ax_incorrect) = plt.subplots(2, sharex=True)
ax_correct.plot(correct["out"], color="blue")
ax_correct.set_title(f"Reconstructed correct") #; MSE {mse_correct}")
ax_incorrect.plot(incorrect["out"], color="blue")
ax_incorrect.set_title(f"Reconstructed incorrect") #; MSE {mse_incorrect}")
fig.tight_layout()
plt.show()
print("Correct MSE: ", mse_correct)
print("Incorrect MSE: ", mse_incorrect)
# def create_plot_auto(correct, incorrect):
#     fig, (ax_correct, ax_incorrect) = plt.subplot(2, sharex=True)
#     ax_correct.plot(correct)

# Load Model

In [ ]:
import os
import torch
from TCN import TCNAE
import utils
import json
import torch.onnx

path = "Results/LOSO_a200_b300_f44_seq43/Models"
model_name = "TCNAE_4"
conditions = ["nod", "gaze", "headAndGaze"]
model_names = ["TCNAE_1_gaze", "TCNAE_1_headAndGaze", "TCNAE_1_nod"]
cond = conditions[0]
# for cond in conditions:
#     with open(os.path.join(path, model_name + f"_{cond}_info.json")) as json_file:
#         model_info = json.load(json_file)
        # print(model_info)
    # model = utils.load_model(os.path.join(path, model_name + f"_{cond}.pth"), TCNAE, model_info["model_parameter"])

for model_name in model_names:
    with open(os.path.join(path, model_name + "_info.json")) as json_file:
        model_info = json.load(json_file)

    model = utils.load_model(os.path.join(path, model_name + ".pth"), TCNAE, model_info["model_parameter"])

    dummy_input = torch.randn(1, 1, model_info["model_parameter"]["seq_len"])
    model.eval()
    # dummy_output = model(dummy_input)
    # print(dummy_output)

    # save model as onnx at the same location like the model_info

    torch.onnx.export(model, dummy_input, os.path.join(path, model_name + ".onnx"))

    print(f"Model has been saved as {model_name}.onnx")

In [ ]:
# Test models
input = torch.randn(5, 1, 1, 1, 1, 1, 1, 61)
output = model(input)

print(output)

# TCNAE final parameters results

In [10]:
import os
import pandas as pd

path = "Results/HyperparameterSearch_TCNAE_a200_b300_f44_seq43/results_log.csv"
df = pd.read_csv(path)
df.rename(columns={
    "correct_acc": "Correct",
    "incorrect_acc": "Incorrect",
    "n_correct": "Correct_samples",
    "n_incorrect": "Incorrect_samples"  
}, inplace=True)
df["Macro"] = (df["Correct"] + df["Incorrect"]) / 2
df["Micro"] = (df["Correct"] * df["Correct_samples"] + df["Incorrect"] * df["Incorrect_samples"]) / (df["Correct_samples"] + df["Incorrect_samples"])
df[["Correct", "Incorrect", "Macro", "Micro"]].mean()

Correct      0.871524
Incorrect    0.890476
Macro        0.881000
Micro        0.871501
dtype: float64

# Statistical power analysis

In [5]:
from statsmodels.stats.power import TTestPower

# Define parameters
effect_sizes = [0.2, 0.33, 0.5, 0.6, 0.7, 0.8]  # Small, medium, large effect sizes
alpha = 0.05  # Significance level
power = 0.95  # Desired statistical power

# Compute required sample sizes for a paired t-test
sample_sizes = {f"d = {d}": int(TTestPower().solve_power(effect_size=d, alpha=alpha, power=power, alternative='two-sided')) 
                for d in effect_sizes}

sample_sizes


{'d = 0.2': 326,
 'd = 0.33': 121,
 'd = 0.5': 53,
 'd = 0.6': 38,
 'd = 0.7': 28,
 'd = 0.8': 22}

In [3]:
import pandas as pd
import numpy as np

# Example DataFrame
data = {
    "Local Gaze Direction X": [0.1, 0.2, 0, 0.4, 0],
    "Local Gaze Direction Y": [0.5, 0.6, 0, 0.8, 0],
    "Local Gaze Direction Z": [0.9, 1.0, 0, 1.2, 0],
    "isValid": [True, True, False, True, False]
}
df = pd.DataFrame(data)

# Replace invalid rows with the last valid gaze direction vector
df.loc[~df["isValid"], ["Local Gaze Direction X", "Local Gaze Direction Y", "Local Gaze Direction Z"]] = np.nan
df[["Local Gaze Direction X", "Local Gaze Direction Y", "Local Gaze Direction Z"]] = df[
    ["Local Gaze Direction X", "Local Gaze Direction Y", "Local Gaze Direction Z"]
].fillna(method="ffill")

display(df)

,Local Gaze Direction X,Local Gaze Direction Y,Local Gaze Direction Z,isValid
0,0.1,0.5,0.9,True
1,0.2,0.6,1.0,True
2,0.2,0.6,1.0,False
3,0.4,0.8,1.2,True
4,0.4,0.8,1.2,False
